In [1]:
import os, sys, time
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
import yaml
from os.path import join
import cPickle as pickle

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions

data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'


start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

nrows = 10000
bin_num = 1 ## number of bins to separate data by start_timeb

In [2]:
tmp_train, tmp_test, bins, bin_names = bosch_functions.create_grouped_index_df(bin_num)

data loading takes  64.7  seconds.


In [3]:
print tmp_train.time_window_num.value_counts(), '\n', tmp_test.time_window_num.value_counts()

0    673861
Name: time_window_num, dtype: int64 
0    674503
Name: time_window_num, dtype: int64


In [4]:
none_selected_window_num = bin_names[:]
skipped_test_row_num = tmp_test.loc[tmp_test['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()
skipped_train_row_num = tmp_train.loc[tmp_train['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()
#bosch_functions.subset_complete_data_by_index(skipped_train_row_num, skipped_test_row_num, train_data_file, test_data_file)

In [5]:
train_cat_cols = pd.read_csv(join(data_path, train_cat_file), index_col=id_column_name, nrows=0)
train_date_cols = pd.read_csv(join(data_path, train_date_file), index_col=id_column_name, nrows=0)
train_num_cols = pd.read_csv(join(data_path, train_num_file), index_col=id_column_name, nrows=0)

In [6]:
### section to obtain the feature impotance from xgboost model
bin_NaN_data_path = '/home/ymm/kaggle/bosch/data_2_bins_xgb_combined_models/data_bin_NaN_models'

def collect_feature_names(data_path, fea_name='feature', thres_name = None, thres = 10):
    csv_files = [f for f in os.listdir(data_path) if '.csv' in f]
    feature_names = set()
    for file_name in csv_files:
        data = pd.read_csv(join(data_path, file_name), index_col=0)
        if thres_name is None:
            feature_names = feature_names.union(data[fea_name])
        else:
            feature_names = feature_names.union(data.loc[data[thres_name] > thres, fea_name])
            
    return feature_names

bin_NaN_selected_col_names = collect_feature_names(bin_NaN_data_path, 'feature', 'fscore', 10)

In [7]:
len(bin_NaN_selected_col_names)

568

In [8]:
print sum(train_num_cols.columns.isin(bin_NaN_selected_col_names)), sum(train_cat_cols.columns.isin(bin_NaN_selected_col_names)), sum(train_date_cols.columns.isin(bin_NaN_selected_col_names))

554 14 0


In [9]:
selected_cat_col_names =  train_cat_cols.columns[train_cat_cols.columns.isin(bin_NaN_selected_col_names)].tolist()
selected_num_col_names =  train_num_cols.columns[train_num_cols.columns.isin(bin_NaN_selected_col_names)].tolist()

In [10]:
selected_cat_col_names.append(id_column_name)
selected_num_col_names.append(id_column_name)

In [11]:
%time train_cat  = pd.read_csv(join(data_path, train_cat_file),  index_col='Id', skiprows=skipped_train_row_num, usecols=selected_cat_col_names)
%time test_cat   = pd.read_csv(join(data_path, test_cat_file),   index_col='Id', skiprows=skipped_train_row_num, usecols=selected_cat_col_names)

CPU times: user 20.8 s, sys: 823 ms, total: 21.6 s
Wall time: 21.6 s
CPU times: user 17.5 s, sys: 1.16 s, total: 18.7 s
Wall time: 18.7 s


In [12]:
%time train_num  = pd.read_csv(join(data_path, train_num_file),  index_col='Id', skiprows=skipped_train_row_num, usecols=selected_num_col_names)
%time test_num   = pd.read_csv(join(data_path, test_num_file),   index_col='Id', skiprows=skipped_train_row_num, usecols=selected_num_col_names)

CPU times: user 36.3 s, sys: 3.23 s, total: 39.6 s
Wall time: 39.6 s
CPU times: user 38.6 s, sys: 1.95 s, total: 40.6 s
Wall time: 40.6 s


In [28]:
print train_num.shape, test_num.shape
train_num.head()

(509886, 554) (509887, 554)


,L0_S8_F144,L0_S12_F330,L0_S12_F332,L0_S12_F334,L0_S12_F336,L0_S12_F338,L0_S12_F340,L0_S12_F342,L0_S12_F344,L0_S12_F346,...,L3_S44_F4121,L3_S45_F4124,L3_S47_F4138,L3_S47_F4143,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S48_F4196,L3_S48_F4198,L3_S50_F4243
Id,,,,,,,,,,,,,,,,,,,,,
6,NaN,0.096,0.076,-0.065,-0.136,0.169,0.231,-0.014,-0.072,-0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,-0.051,-0.082,-0.181,-0.481,0.113,0.154,-0.010,-0.072,-0.054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,-0.008,-0.003,-0.065,-0.136,-0.053,-0.154,-0.006,-0.041,-0.049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,0.041,0.062,0.081,0.140,0.002,0.000,0.002,-0.010,0.223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
print train_cat.shape
train_cat.head()

(509886, 14)


,L1_S24_F675,L1_S24_F1510,L1_S24_F1582,L1_S24_F1584,L1_S24_F1585,L1_S25_F1852,L1_S25_F2099,L1_S25_F2779,L2_S26_F3038,L2_S27_F3131,L3_S29_F3317,L3_S29_F3475,L3_S32_F3851,L3_S32_F3854
Id,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
16,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN


In [15]:
combined_cat = pd.concat([train_cat, test_cat])

In [16]:
combined_cat = combined_cat.astype(str)

In [17]:
oneHot_combined_cat = pd.get_dummies(combined_cat, dummy_na=True)

In [18]:
oneHot_combined_cat.head()

,L1_S24_F675_1.0,L1_S24_F675_2.0,L1_S24_F675_3.0,L1_S24_F675_4.0,L1_S24_F675_5.0,L1_S24_F675_nan,L1_S24_F675_nan,L1_S24_F1510_1.0,L1_S24_F1510_2.0,L1_S24_F1510_3.0,...,L3_S32_F3854_4.0,L3_S32_F3854_48.0,L3_S32_F3854_492.0,L3_S32_F3854_512.0,L3_S32_F3854_55424.0,L3_S32_F3854_63616.0,L3_S32_F3854_65536.0,L3_S32_F3854_8.0,L3_S32_F3854_nan,L3_S32_F3854_nan
Id,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
41,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
train_index = train_cat.index
test_index  = test_cat.index

In [34]:
oneHot_train_cat = oneHot_combined_cat.ix[train_index]
oneHot_test_cat  = oneHot_combined_cat.ix[test_index]

In [25]:
print oneHot_train_cat.shape
oneHot_train_cat.head()

(509886, 102)


,L1_S24_F675_1.0,L1_S24_F675_2.0,L1_S24_F675_3.0,L1_S24_F675_4.0,L1_S24_F675_5.0,L1_S24_F675_nan,L1_S24_F675_nan,L1_S24_F1510_1.0,L1_S24_F1510_2.0,L1_S24_F1510_3.0,...,L3_S32_F3854_4.0,L3_S32_F3854_48.0,L3_S32_F3854_492.0,L3_S32_F3854_512.0,L3_S32_F3854_55424.0,L3_S32_F3854_63616.0,L3_S32_F3854_65536.0,L3_S32_F3854_8.0,L3_S32_F3854_nan,L3_S32_F3854_nan
Id,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
41,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
combined_train = pd.merge(oneHot_train_cat, train_num, how='outer', left_index=True, right_index=True)
combined_test  = pd.merge(oneHot_test_cat,  test_num,  how='outer', left_index=True, right_index=True)

In [36]:
print combined_train.shape, combined_test.shape

(509886, 656) (509887, 656)


In [39]:
combined_train.to_csv('bosch_processed_selected_NaN_train_data.csv')
combined_test.to_csv('bosch_processed_selected_NaN_test_data.csv')

In [37]:
combined_train.head()

,L1_S24_F675_1.0,L1_S24_F675_2.0,L1_S24_F675_3.0,L1_S24_F675_4.0,L1_S24_F675_5.0,L1_S24_F675_nan,L1_S24_F675_nan,L1_S24_F1510_1.0,L1_S24_F1510_2.0,L1_S24_F1510_3.0,...,L3_S44_F4121,L3_S45_F4124,L3_S47_F4138,L3_S47_F4143,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S48_F4196,L3_S48_F4198,L3_S50_F4243
Id,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
combined_test.head()

,L1_S24_F675_1.0,L1_S24_F675_2.0,L1_S24_F675_3.0,L1_S24_F675_4.0,L1_S24_F675_5.0,L1_S24_F675_nan,L1_S24_F675_nan,L1_S24_F1510_1.0,L1_S24_F1510_2.0,L1_S24_F1510_3.0,...,L3_S44_F4121,L3_S45_F4124,L3_S47_F4138,L3_S47_F4143,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S48_F4196,L3_S48_F4198,L3_S50_F4243
Id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
for column in tmp_train_cat.columns:
    print tmp_train_cat[column].value_counts()

3.0    21368
2.0     1813
4.0      191
1.0       17
5.0        7
Name: L1_S24_F675, dtype: int64
3.0    121130
2.0      8575
4.0      1449
1.0       174
5.0        78
Name: L1_S24_F1510, dtype: int64
1.0    136
Name: L1_S24_F1582, dtype: int64
1.0    65819
Name: L1_S24_F1584, dtype: int64
1372.0       64
262144.0     34
24.0         20
8389632.0     9
748928.0      2
618624.0      2
83888.0       2
8651776.0     1
8768.0        1
16.0          1
Name: L1_S24_F1585, dtype: int64
4.0    41376
5.0    10156
3.0     2486
6.0      604
2.0      243
7.0      185
1.0       32
8.0       21
Name: L1_S25_F1852, dtype: int64
1.0    5303
Name: L1_S25_F2099, dtype: int64
4.0    11500
5.0     4942
3.0      965
6.0      254
2.0       67
7.0       30
1.0       12
8.0        1
Name: L1_S25_F2779, dtype: int64
1.0    180002
Name: L2_S26_F3038, dtype: int64
1.0    78356
Name: L2_S27_F3131, dtype: int64
1.0    341597
Name: L3_S29_F3317, dtype: int64
1.0    327624
Name: L3_S29_F3475, dtype: int64
1.0    8958

In [59]:
#train_date = pd.read_csv(data_path + train_date_file, index_col='Id', skiprows=skipped_train_row_num, nrows = nrows)
train_num  = pd.read_csv(data_path + train_num_file,  index_col='Id', skiprows=skipped_train_row_num, nrows = nrows)
train_cat  = pd.read_csv(data_path + train_cat_file,  index_col='Id', skiprows=skipped_train_row_num, nrows = nrows)

In [60]:
train_cat[dep_var_name] = train_num[dep_var_name]

In [44]:
for column in train_cat.columns:
    if len(train_cat[column].value_counts().keys()) > 2:
        print column

L1_S24_F675
L1_S24_F705
L1_S24_F710
L1_S24_F1114
L1_S24_F1137
L1_S24_F1140
L1_S24_F1510
L1_S24_F1525
L1_S24_F1530
L1_S24_F1585
L1_S24_F1589
L1_S24_F1593
L1_S24_F1598
L1_S24_F1603
L1_S24_F1608
L1_S24_F1613
L1_S24_F1617
L1_S24_F1621
L1_S24_F1626
L1_S24_F1631
L1_S24_F1636
L1_S24_F1641
L1_S24_F1646
L1_S24_F1651
L1_S24_F1656
L1_S24_F1661
L1_S24_F1666
L1_S24_F1671
L1_S25_F1852
L1_S25_F2229
L1_S25_F2496
L1_S25_F2779
L3_S32_F3854


In [61]:
tmp_train_cat = train_cat.copy()

In [62]:
col_name = 'L1_S24_F710'
#train_cat[[col_name, dep_var_name]].groupby(col_name).mean()
#print train_cat[col_name].value_counts()
print tmp_train_cat[col_name].isnull().sum()

9586


In [63]:
tmp_train_cat[col_name] = tmp_train_cat[col_name].fillna('missing')
dict_results = tmp_train_cat[col_name].value_counts().to_dict()

In [64]:
dict_results

{2.0: 35, 3.0: 373, 4.0: 6, 'missing': 9586}

In [84]:
mean_dep_var_value = tmp_train_cat[[col_name, dep_var_name]].groupby(col_name).mean()[dep_var_name]

In [85]:
dep_var_dict = mean_dep_var_value.to_dict()